In [3]:
library(stringr)
library(glue)
library(purrr)
library(tidyverse)
library(tictoc)

In [7]:
#Get the statistical results for a specific dataset and cutoff

# INPUTS:
# data: a data matrix in the format of "complete_genome_SARS-CoV-2_info_k={i}.txt", where i: the length of each k-mer
# colname: the name of the column where the cut will be applied [works for "Value" & "Godel_number"]
# pts: the total number of different cutoffs [also the number of points to be plotted]

# OUTPUTS: a [3*points] matrix of [0,1 or -1] depending on the p-values of the three statistical tests
#(wilcoxon, fisher, kolmogorov-smirnov) for the data set combinations: Low-High, Low-Total, High-Total

#NULL HYPOTHESIS FOR EACH TEST(X,Y) <= [p > 0.05]
#wilcoxon: both x and y have same means
#fisher:  both x and y have same variances
#kolmogorov-smirnov:  both x and y have the same distribution

#1: if all three tests return p-values < 0.05 [which means that the datasets have different means, variances & distributions]
#0: if at least one of the tests returns p-value > 0.05
#-1: if any of the tests cannot be performed because one of the datasets has one or zero elements
#1:[similar], 0:[different], -1:[cannot compute <= segment has 1 or 0 elements]

stats_cutoff<- function(data, colname, pts){
    
    #DEFINE THE DEFAULTS
    if(missing(colname)){
        colname = 'Value'
    }
    if (missing(pts)){
        pts = 9
    }
    
    #CHOOSE METHOD
    column_cut <- select(data, all_of(colname))
    
    min_col = min(column_cut)
    max_col = max(column_cut)
    cuts = pts + 2 - 1
    seg = (max_col - min_col)/cuts
    seq_cuts = min_col + seg*(1:pts)   
    
    wilcoxon <- matrix(, nrow = 3, ncol = pts, dimnames = list(c('Low-High', 'Low-Total', 'High-Total'), seq_cuts))
    fisher <- matrix(, nrow = 3, ncol = pts, dimnames = list(c('Low-High', 'Low-Total', 'High-Total'), seq_cuts))
    kolm_smirnov <- matrix(, nrow = 3, ncol = pts, dimnames = list(c('Low-High', 'Low-Total', 'High-Total'), seq_cuts))
    stats <- matrix(, nrow = 3, ncol = pts, dimnames = list(c('Low-High', 'Low-Total', 'High-Total'), seq_cuts))
    
    #STATISTICAL TESTS [WILCOXON, FISHER, KOLMOGOROV-SMIRNOV]
    for (cut in 1:pts){
        
        #MAKE THE CUT
        low_cut <- data[column_cut < seq_cuts[cut], ]
        high_cut <- data[column_cut >= seq_cuts[cut], ]
        
        if (colname == 'Value'){
            low = table(low_cut[, 2])
            high = table(high_cut[, 2])
            total = table(data[, 2])
        }
        if (colname == 'Godel_number'){
            low = low_cut[, 3]
            high = high_cut[, 3]
            total = data[, 3]
        }        
        
        #COMPUTE THE P-VALUES
        #Low-High
        if (length(low)>1 & length(high)>1) {
            wilcoxon[1, cut] = wilcox.test(low, high)$p.value
            fisher[1, cut] = var.test(low, high)$p.value
            kolm_smirnov[1, cut] = ks.test(low, high)$p.value
        }
        else {
            wilcoxon[1, cut] = -1
            fisher[1, cut] = -1
            kolm_smirnov[1, cut] = -1
        }
            
        #Low-Total
        if (length(low)>1 & length(total)>1) {
            wilcoxon[2, cut] = wilcox.test(low, total)$p.value
            fisher[2, cut] = var.test(low, total)$p.value
            kolm_smirnov[2, cut] = ks.test(low, total)$p.value
        }
        else {
            wilcoxon[2, cut] = -1
            fisher[2, cut] = -1
            kolm_smirnov[2, cut] = -1
        }
        
        #High-Total 
        if (length(high)>1 & length(total)>1) {
            wilcoxon[3, cut] = wilcox.test(high, total)$p.value
            fisher[3, cut] = var.test(high, total)$p.value
            kolm_smirnov[3, cut] = ks.test(high, total)$p.value
        }
        else {
            wilcoxon[3, cut] = -1
            fisher[3, cut] = -1
            kolm_smirnov[3, cut] = -1
        }
    }
        
    #COMPUTE THE STATISTICAL IMPORTANCE
    #Wlicoxon
    wlc = wilcoxon
    wlc[wlc>= 0.05] <- 1
    wlc[wlc<0.05 & wlc>=0] <- 0

    #Fisher
    fsh = fisher
    fsh[fsh>= 0.05] <- 1
    fsh[fsh<0.05 & fsh>=0] <- 0

    #Kolmogorov-Smirnov
    klm = kolm_smirnov
    klm[klm>= 0.05] <- 1
    klm[klm<0.05 & klm>=0] <- 0

    stats = wlc*fsh*klm
        
    return (list(stats, seq_cuts))
}

In [8]:
#Main
tic()
options(warn=-1)


#2 CHANGES TO BE MADE IF RUNNING ON SERVER

#Define paths
wd_path = file.path('C:','Users', 'user', 'Desktop', 'Workspaces', 'R', 'INAB')
#1/2!TO-DO: Change wd_path to whatever is necessary if running on server
res_path = file.path(wd_path, 'results')
txt_path = file.path(res_path, 'txt')
img_path = file.path(res_path, 'img')

#Create directories
invisible(ifelse(!dir.exists(file.path(txt_path)), dir.create(file.path(txt_path)), FALSE))
invisible(ifelse(!dir.exists(file.path(img_path)), dir.create(file.path(img_path)), FALSE))

#Choose number of points - pts
pts = 20

#Choose length of kmers - k
for (k in c(10, 15, 20)){
    
    #Load data
    data_path = file.path(wd_path,'data', glue('sars_1000_info_k={k}.txt'))
    #2/2!TO-DO: Change 'sars_1000_info_k={k}.txt' to 'complete_genome_SARS-CoV-2_info_k={k}.txt' if running on server
    data <- read.table(data_path)

    #Filter data for 'ACTG'
    filt_data <- data[str_detect(data[, 1], '^[ACTG]+$'),]

    #Multiplicity
    res_mult = stats_cutoff(filt_data, colname = 'Value', pts)

    stats_mult = res_mult[1]
    cuts_mult = unlist(res_mult[2])
    
    #Godel Numbers
    res_godel = stats_cutoff(filt_data, colname = 'Godel_number', pts)
    stats_godel = res_godel[1]
    cuts_godel = unlist(res_godel[2])

    #Save results
    write.csv(stats_mult, glue('{txt_path}/total_stats_mult_k={k}_pts={pts}'))
    write.csv(stats_godel, glue('{txt_path}/total_stats_godel_k={k}_pts={pts}'))

    #Plot
    y_lim = c(-2, 2)

    #Multiplicity
    png(glue('{img_path}/total_stats_mult_k={k}.png'))
    par(mfcol=c(3,1))

    x_mult = cuts_mult
    df_mult = as.data.frame(stats_mult)

    plot(x =  x_mult, y = unlist(df_mult[1, ]), type = 'o', main = 'Low-High', xlab = 'cutoff', ylab = 'Importance', ylim = y_lim)
    abline(h = 0)
    plot(x =  x_mult, y = unlist(df_mult[2, ]), type = 'o', main = 'Low-Total', xlab = 'cutoff', ylab = 'Importance', ylim = y_lim)
    abline(h = 0)
    plot(x =  x_mult, y = unlist(df_mult[3, ]), type = 'o', main = 'High-Total', xlab = 'cutoff', ylab = 'Importance', ylim = y_lim)
    abline(h = 0)
    
    dev.off()

    #Godel Numbers
    png(glue('{img_path}/total_stats_godel_k={k}.png'))
    par(mfcol=c(3,1))

    x_godel = cuts_godel
    df_godel = as.data.frame(stats_godel)

    plot(x =  x_godel, y = unlist(df_godel[1, ]), type = 'o', main = 'Low-High', xlab = 'cutoff', ylab = 'Importance', ylim = y_lim)
    abline(h = 0)
    plot(x =  x_godel, y = unlist(df_godel[2, ]), type = 'o', main = 'Low-Total', xlab = 'cutoff', ylab = 'Importance', ylim = y_lim)
    abline(h = 0)
    plot(x =  x_godel, y = unlist(df_godel[3, ]), type = 'o', main = 'High-Total', xlab = 'cutoff', ylab = 'Importance', ylim = y_lim)
    abline(h = 0)
    
    dev.off()
}
toc()

63.51 sec elapsed
